In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 83.3 kB in 2s (54.8 kB/s)
Reading package lists... Done


In [ ]:
# Start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('VineAnalysis').getOrCreate()

In [ ]:
# Extract and load Amazon data into a spark DataFrame
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [20]:
# Recreate the vine_table
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [26]:
# Filter data and create a new table to retrieve all the rows where the total_votes count is equal to or greater than 20
filt_vine_df = vine_df.filter(vine_df["total_votes"] >= 20)
filt_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [27]:
# Filter the table above and create a new  table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
#new_vine_df = filt_vine_df.filter(filt_vine_df['helpful_votes']/filt_vine_df['total_votes'] >= 0.5)
#new_vine_df.show()

new_vine_df = filt_vine_df.filter(filt_vine_df["helpful_votes"]/filt_vine_df["total_votes"] >= 0.5)
new_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

In [28]:
# Filter the table created above and create a new table that retrieves all the rows where a review was written as part of the Vine program (paid)
vine_verified = new_vine_df.filter(new_vine_df["vine"] == 'Y')
vine_verified.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|
|R1M3LATUI38MK0|          5|           46|         50|   Y|                N|
|R20JZQ3AQVYNAU|          4|          103|        113|   Y|                N|
|R1EHXYTMBO4D28|          1|           56|         61|   Y|     

In [30]:
# Retrieve all the rows where the review was not part of the Vine program (unpaid)
vine_unverified = new_vine_df.filter(new_vine_df["vine"] == 'N')
vine_unverified.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|                Y|
| RYA9ESIATW4EC|          1|           23|         27|   N|     

In [32]:
# Determine the total number of paid reviews
from pyspark.sql.functions import count
paid_df = vine_df.filter(vine_df["vine"] == 'Y')
paid_df.select(count(paid_df.star_rating)).show()

+------------------+
|count(star_rating)|
+------------------+
|             32026|
+------------------+



In [33]:
# Determine the total number of unpaid reviews
unpaid_df = vine_df.filter(vine_df["vine"] == 'N')
unpaid_df.select(count(unpaid_df.star_rating)).show()

+------------------+
|count(star_rating)|
+------------------+
|           5299189|
+------------------+



In [35]:
# Determine the total the number of 5-star paid reviews
paid_df.filter("star_rating = 5").count()

13127

In [36]:
# Determine the total the number of 5-star unpaid reviews
unpaid_df.filter("star_rating = 5").count()

3344959

In [38]:
# Determine the percentage of 5-star paid reviews
paid_percentage = (13127/32026)*100
print(paid_percentage)

40.98857178542434


In [39]:
# Determine the percentage of 5-star unpaid reviews
unpaid_percentage = (3344959/5299189)*100
print(unpaid_percentage)

63.12209283345055
